# Frequency Analysis — Laboratory Report (ELA24)

Author: Ella Elektronik  
Performed by: Ella Elektronik, Inge Instrument  
Supervisor: Lennart Lärare  
Lab date: 1900-01-01  
Report date: 1900-01-08  

Institution: Yrgo — Course: Signals and Sensors (ELA24)


## Abstract
Provide a brief, self-contained summary: aim of the lab, methods used, key results, and conclusions. The abstract helps the reader quickly understand the content and findings of the report.


## Table of Contents
1. Introduction  
2. Experiment  
3. Results  
4. Discussion  
5. Conclusions  
6. References  
Appendices


In [13]:
# Utility: Load FFT CSV exported by Keysight/DSOX (semicolon delimiter, comma decimals)
# Returns a DataFrame with columns: freq_hz, magnitude (Vrms), and dBVrms
from __future__ import annotations
import pandas as pd
import numpy as np
from pathlib import Path


def load_fft_csv(path: str | Path, *, add_db: bool = True) -> pd.DataFrame:
    """
    Load an FFT CSV with Keysight export format (RND_lab_*, rc_fft_*, trace_*_fft.csv).

    - Delimiter: ';'
    - Decimal separator: ','
    - Metadata lines may precede numeric data; numeric data ends up in the last two columns.

    Returns a DataFrame with:
        freq_hz (float), magnitude (Vrms, float), and optionally dBVrms (20*log10(Vrms)).
    """
    path = Path(path)
    # Use python engine for flexible parsing; do not pass low_memory (unsupported by python engine)
    df = pd.read_csv(path, sep=';', decimal=',', header=None, engine='python')

    # Convert all columns to numeric where possible
    num = df.apply(pd.to_numeric, errors='coerce')

    # Identify columns that are mostly numeric; keep the last two such columns
    numeric_cols = [c for c in num.columns if num[c].notna().sum() > 0]
    if len(numeric_cols) < 2:
        raise ValueError(f"Could not find two numeric columns in {path}")

    c2, c1 = numeric_cols[-1], numeric_cols[-2]  # last two numeric columns
    out = num[[c1, c2]].rename(columns={c1: 'freq_hz', c2: 'magnitude'})

    # Clean rows: drop NaNs and non-positive frequencies
    out = out.dropna()
    out = out[out['freq_hz'] > 0]

    if add_db:
        # Guard against zeros/negatives for dB conversion; CSV magnitudes are Vrms
        mag = out['magnitude'].clip(lower=np.finfo(float).eps)
        out['dBVrms'] = 20 * np.log10(mag)
        # Back-compat alias if any legacy cell expects this name
        out['magnitude_db'] = out['dBVrms']

    # Sort by frequency just in case
    out = out.sort_values('freq_hz').reset_index(drop=True)
    return out


# Tiny smoke check (no execution here):
# d = load_fft_csv('RND_lab_sine_1khz_fft.csv')
# d.head()

In [14]:
# (Appendix/example) RND AWG — Sine (~1 kHz) — Spectrum (dBVrms)
# This example is kept for reference. It is disabled to avoid cluttering the main results.
# Uncomment to use in Appendix.
if False:
    import matplotlib.pyplot as plt
    fft = load_fft_csv('RND_lab_sine_1khz_fft.csv', add_db=True)
    plt.figure(figsize=(10, 4))
    plt.plot(fft['freq_hz'], fft['dBVrms'], lw=1.0)
    plt.title('RND AWG — Sine (~1 kHz) — Spectrum (dBVrms)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dBVrms)')
    plt.grid(True, which='both', ls=':', alpha=0.5)
    plt.tight_layout()
    plt.show()

### Essential plots in this report
- Theory vs measurement (first N harmonics, dBVrms): Sawtooth, Triangle, Sine — using `compare_plot(...)` with the Keysight traces.
- Generator comparison (RND vs Keysight): Sawtooth only, shown separately with shared axes.

The RND sine/triangle full-spectrum plots are not shown in the main flow to keep focus on the required comparisons.

In [15]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple


def to_dbv(df: pd.DataFrame, floor_vrms: float = 1e-12) -> pd.DataFrame:
    """Convert magnitude (assumed RMS voltage) to dBVrms (20*log10(Vrms/1V))."""
    d = df.copy()
    mag = d['magnitude'].clip(lower=floor_vrms)
    d['dBVrms'] = 20 * np.log10(mag)
    return d


def find_fundamental_dbv(d: pd.DataFrame, fmin: float = 1.0) -> Tuple[float, float]:
    """Return (f0, dBVrms0) for max amplitude above fmin."""
    dd = d[d['freq_hz'] >= fmin]
    i = dd['dBVrms'].idxmax()
    return float(dd.loc[i, 'freq_hz']), float(dd.loc[i, 'dBVrms'])


def metric_thd_db(d_raw: pd.DataFrame, f0: float, nharm: int = 10) -> float:
    """THD in dB using RMS amplitudes: 20log10( sqrt(sum_{k>=2} Ak^2) / A1 )."""
    # Use linear Vrms for THD
    A1 = d_raw[d_raw['freq_hz'].between(0.98 * f0, 1.02 * f0)]['magnitude'].max()
    A1 = float(max(A1, np.finfo(float).eps))
    thd_power = 0.0
    for k in range(2, nharm + 1):
        fk = k * f0
        Ak = d_raw[d_raw['freq_hz'].between(0.995 * fk, 1.005 * fk)]['magnitude'].max()
        Ak = float(max(Ak, 0.0))
        thd_power += Ak ** 2
    thd_ratio = (thd_power ** 0.5) / A1
    thd_db = 20 * np.log10(max(thd_ratio, np.finfo(float).eps))
    return float(thd_db)


def metric_sfdr_dbv(d: pd.DataFrame, f0: float) -> float:
    """SFDR in dB relative to fundamental in dBVrms: spur_peak_dBVrms - fundamental_dBVrms (negative)."""
    fund = d[d['freq_hz'].between(0.99 * f0, 1.01 * f0)]['dBVrms'].max()
    mask = ~d['freq_hz'].between(0.99 * f0, 1.01 * f0)
    spur = d.loc[mask, 'dBVrms'].max()
    return float(spur - fund)


def metric_noise_floor_dbv(d: pd.DataFrame, f0: float) -> float:
    """Median dBVrms excluding fundamental and first 10 harmonics windows."""
    m = ~d['freq_hz'].between(0.99 * f0, 1.01 * f0)
    for k in range(2, 11):
        fk = k * f0
        m &= ~d['freq_hz'].between(0.995 * fk, 1.005 * fk)
    return float(d.loc[m, 'dBVrms'].median())


def summarize_metrics_dbv(df: pd.DataFrame) -> Dict[str, float]:
    d = to_dbv(df)
    f0, d0 = find_fundamental_dbv(d)
    return {
        'f0_hz': f0,
        'fund_dBVrms': d0,
        'THD_dB': metric_thd_db(df, f0),
        'SFDR_dB': metric_sfdr_dbv(d, f0),
        'NoiseFloorMed_dBVrms': metric_noise_floor_dbv(d, f0),
    }

In [16]:
import matplotlib.pyplot as plt
from textwrap import dedent


def plot_single_source(title: str, df_dbv, xlim=None, ylim=None):
    plt.figure(figsize=(10, 4))
    plt.plot(df_dbv['freq_hz'], df_dbv['dBVrms'], alpha=0.95)
    if xlim is None:
        xlim = (0, df_dbv['freq_hz'].max() * 0.2)
    plt.xlim(*xlim)
    if ylim is None:
        ymax = df_dbv['dBVrms'].max()
        ylim = (max(-200, ymax - 120), ymax + 5)
    plt.ylim(*ylim)
    plt.title(f"{title} — Spectrum (dBVrms)")
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude (dBVrms)')
    plt.grid(True, ls=':', alpha=0.5)
    plt.tight_layout()
    plt.show()


def _common_axes(d1, d2, f0_hint=None, n_harm=10):
    # Common x-axis: show up to ~10 harmonics if possible, bounded by both spectra span
    if f0_hint is None:
        # Estimate fundamentals from dBVrms peaks
        def est_f0(d):
            i = d['dBVrms'].idxmax()
            return float(d.loc[i, 'freq_hz'])
        f0 = 0.5 * (est_f0(d1) + est_f0(d2))
    else:
        f0 = float(f0_hint)
    target_xmax = n_harm * f0
    xmax = min(d1['freq_hz'].max(), d2['freq_hz'].max(), target_xmax)
    xlim = (0.0, max(1.0, xmax))

    # Common y-axis anchored to the larger peak
    ypeak = max(d1['dBVrms'].max(), d2['dBVrms'].max())
    ylim = (max(-200, ypeak - 120), ypeak + 5)
    return xlim, ylim


def compare_sources_separate(title: str, rnd_path: str, key_path: str):
    rnd_raw = load_fft_csv(rnd_path, add_db=True)
    key_raw = load_fft_csv(key_path, add_db=True)

    rnd = rnd_raw[['freq_hz', 'dBVrms']].copy()
    key = key_raw[['freq_hz', 'dBVrms']].copy()

    xlim, ylim = _common_axes(rnd, key, n_harm=10)

    # Separate plots with shared axes (dBVrms only)
    plot_single_source(f"{title} — RND AWG", rnd, xlim=xlim, ylim=ylim)
    plot_single_source(f"{title} — Keysight FG", key, xlim=xlim, ylim=ylim)

    # Minimal metrics print (dBVrms-based)
    m_rnd = summarize_metrics_dbv(rnd_raw.rename(columns={'magnitude': 'magnitude'}))
    m_key = summarize_metrics_dbv(key_raw.rename(columns={'magnitude': 'magnitude'}))

    def one_line(m):
        return f"f0={m['f0_hz']:.2f} Hz, Fundamental={m['fund_dBVrms']:.1f} dBVrms, THD={m['THD_dB']:.1f} dB, SFDR={m['SFDR_dB']:.1f} dB"

    print(f"RND AWG — {title}: {one_line(m_rnd)}")
    print(f"Keysight FG — {title}: {one_line(m_key)}")


def compare_two_files_separate(title_base: str, path_a: str, label_a: str, path_b: str, label_b: str, f0_hint: float | None = None):
    a_raw = load_fft_csv(path_a, add_db=True)
    b_raw = load_fft_csv(path_b, add_db=True)
    a = a_raw[['freq_hz', 'dBVrms']].copy()
    b = b_raw[['freq_hz', 'dBVrms']].copy()
    xlim, ylim = _common_axes(a, b, f0_hint=f0_hint, n_harm=10)
    plot_single_source(f"{title_base} — {label_a}", a, xlim=xlim, ylim=ylim)
    plot_single_source(f"{title_base} — {label_b}", b, xlim=xlim, ylim=ylim)

# Note: Calls removed here. See the dedicated Sawtooth-only cell below for the actual comparison.


## 1. Introduction
Signals can be represented in both time and frequency domains. Using Fourier series, periodic signals are decomposed into sinusoidal components whose amplitudes decay depending on waveform sharpness. This lab studies the harmonic content of sawtooth, triangle, and sine signals and the effect of an RC low‑pass filter on their spectra and time waveforms.


### Tasks (as executed)
1. Sawtooth Fourier series (1 kHz, 5 Vpp, 0 V offset): measure first 10 harmonics (1–10 kHz) with FFT; compare against theory.
2. Triangle Fourier series: measure first 10 harmonics with same settings; compare against theory (odd n expected).
3. Sine: measure first harmonics; discuss how many tones are expected and why others may appear.
4. RC‑filtered sawtooth: insert RC with R=8.2 kΩ, C=10 nF; measure first 10 output harmonics and compare to theory × |H(jω)|; also inspect the time‑domain waveform.
5. Extra: Compare RND AWG vs Keysight FG for sawtooth at ~1 kHz.

Note on units: The oscilloscope FFT reports dBVrms; conversions and theory are handled in the code cells.


### Learning objectives
- Understand dual representations of periodic signals in time and frequency domains.
- Use Fourier series to quantify harmonic content for sawtooth, triangle, and sine.
- Measure spectra with an oscilloscope FFT (in dBVrms) and compare to theory.
- Analyze the effect of an RC low‑pass filter on amplitude and phase in both domains.
- Practice traceability by saving instrument settings, versions, and screenshots.


## Reading guide (overview)
- Follow the Table of Contents. Run code cells in order where requested in Results.
- Theory notes and supporting code are collected in the Appendices.


### Key formulas (from course notes)

- Sawtooth (all harmonics):  
  $A_n = \dfrac{2A}{\pi n},\; n \ge 1$
- Triangle (odd harmonics):  
  $A_n = \dfrac{8A}{\pi^2 n^2},\; n = 1,3,5,\dots$
- Sine:  
  $A_1 = A,\; A_{n>1}=0$
- RMS and dBV:  
  $V_{\mathrm{rms}} = \dfrac{V_{\mathrm{peak}}}{\sqrt{2}},\; \mathrm{dBV} = 20\log_{10}(V_{\mathrm{rms}}/1\,\mathrm{V})$
- RC low‑pass transfer function:  
  $|H(f)| = \dfrac{1}{\sqrt{1+(f/f_c)^2}},\; \angle H(f) = -\arctan(f/f_c),\; f_c = \dfrac{1}{2\pi RC}$

Note: Theoretical values shown in tables/figures are computed directly from these expressions.


## 2. Experiment

### Equipment
- Signal generator: 1 kHz, 5 Vpp, 0 V DC offset.  
- Oscilloscope: Keysight/Tektronix (FFT mode, Hann window, dBVrms).  
- RC filter:  
  $ R = 8.2\,\text{k}\Omega, \; C = 10\,\text{nF} $  
  Cutoff frequency:  
  $ f_c = \dfrac{1}{2\pi RC} \approx 1.94\,\text{kHz} $
- Software: Python (NumPy, Pandas, Matplotlib).

### Method
1. Measure the first 10 harmonics for sawtooth (1–10 kHz) in the FFT.  
2. Repeat for triangle (odd harmonics expected).  
3. Repeat for sine (fundamental only).  
4. Insert the RC filter and measure the sawtooth output spectrum.  
5. Inspect the time‑domain effect of filtering.  
6. Compare measurements with theoretical Fourier coefficients and RC transfer function.

Traceability: Include filenames and instrument/software versions to allow reproducibility.


In [17]:
# === ELA24: Comparisons in dBVrms (no files written) ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (7.6, 4.6)

# ----- Lab params (edit to match your setup) -----
F0      = 1000.0          # [Hz] fundamental
VPP     = 5.0             # [Vpp] generator-level for theory
A       = VPP/2.0         # [Vpeak]
N_HARM  = 10              # compare first N harmonics
R, C    = 8200.0, 10e-9   # RC low-pass values (8.2k, 10nF)
FC      = 1.0/(2*np.pi*R*C)

# ----- Helpers -----
def vrms_to_dBVrms(vrms, floor=1e-12):
    v = np.maximum(np.asarray(vrms, float), floor)
    return 20.0*np.log10(v)

def rc_mag(f, fc=FC):
    f = np.asarray(f, float)
    return 1.0/np.sqrt(1.0 + (f/fc)**2)

# ----- Fourier theory -> dBVrms -----
def sawtooth_dBVrms(n, A=A):
    n = np.asarray(n, int)
    vpk = 2.0*A/(np.pi*np.maximum(n,1))
    vrms = vpk/np.sqrt(2.0)
    return vrms_to_dBVrms(vrms)

def triangle_dBVrms(n, A=A):
    n = np.asarray(n, int)
    vrms = np.zeros_like(n, float)
    odd = (n % 2 == 1)
    vpk_odd = 8.0*A/(np.pi**2 * np.maximum(n[odd],1)**2)
    vrms[odd] = vpk_odd/np.sqrt(2.0)
    dB = vrms_to_dBVrms(vrms)
    dB[~odd] = np.nan   # even harmonics absent → NaN (not plotted)
    return dB

def sine_dBVrms(n, A=A):
    n = np.asarray(n, int)
    vrms = np.zeros_like(n, float)
    vrms[n == 1] = A/np.sqrt(2.0)   # only fundamental
    dB = vrms_to_dBVrms(vrms)
    dB[n != 1] = np.nan             # others absent
    return dB


def theory_dBVrms(F0=F0, N=N_HARM, waveform='saw', apply_rc=False):
    n = np.arange(1, N+1)
    if waveform == 'saw':
        dB = sawtooth_dBVrms(n)
    elif waveform == 'tri':
        dB = triangle_dBVrms(n)
    elif waveform == 'sin':
        dB = sine_dBVrms(n)
    else:
        raise ValueError("waveform must be 'saw' | 'tri' | 'sin'")
    if apply_rc:
        dB += vrms_to_dBVrms(rc_mag(n*F0))  # add 20*log10|H|
    return pd.DataFrame({'n': n, 'f_Hz': n*F0, 'theory_dBVrms': dB})

# ----- Scope CSV (Vrms) -> spectrum (dBVrms) -----
def read_scope_fft_vrms(csv_path):
    """
    Robust reader: handles semicolon sep, decimal comma, headers/garbage.
    Returns columns: freq[Hz], Vrms, dBVrms
    """
    raw = pd.read_csv(csv_path, sep=';', header=None, engine='python', dtype=str)
    freqs, mags = [], []
    for _, row in raw.iterrows():
        nums = []
        for cell in row.dropna():
            s = cell.strip()
            if not s: 
                continue
            s = s.replace(',', '.')
            try:
                nums.append(float(s))
            except ValueError:
                pass
        if len(nums) >= 2:
            f, v = nums[-2], nums[-1]  # assume last two numeric fields = freq, Vrms
            if f >= 0 and v >= 0:
                freqs.append(f); mags.append(v)
    spec = pd.DataFrame({'freq': freqs, 'Vrms': mags}).sort_values('freq')
    spec = spec.drop_duplicates(subset=['freq']).reset_index(drop=True)
    if spec.empty:
        raise ValueError(f"No numeric data parsed from {csv_path}")
    spec['dBVrms'] = vrms_to_dBVrms(spec['Vrms'].values)
    return spec

def pick_harmonic_bins(spec_df, F0, N_HARM):
    """Nearest bin to n·F0 for n=1..N within spectrum span."""
    freqs = spec_df['freq'].values
    n_all = np.arange(1, N_HARM+1)
    targets = n_all*F0
    ok = (targets >= freqs[0]) & (targets <= freqs[-1])
    n = n_all[ok]; targets = targets[ok]
    idx = np.searchsorted(freqs, targets)
    idx = np.clip(idx, 1, len(freqs)-1)
    chosen = np.where(np.abs(freqs[idx]-targets) < np.abs(freqs[idx-1]-targets), idx, idx-1)
    return pd.DataFrame({
        'n': n,
        'f_target_Hz': targets,
        'f_bin_Hz': freqs[chosen],
        'meas_dBVrms': spec_df['dBVrms'].values[chosen],
    })

# ----- One-shot comparison plot -----
def compare_plot(csv_path, waveform, apply_rc_to_theory=False, title=None,
                 F0=F0, N=N_HARM):
    spec = read_scope_fft_vrms(csv_path)
    meas = pick_harmonic_bins(spec, F0, N)
    theo = theory_dBVrms(F0=F0, N=N, waveform=waveform, apply_rc=apply_rc_to_theory)
    df = meas.merge(theo, on='n', how='left')
    df['err_dB'] = df['meas_dBVrms'] - df['theory_dBVrms']

    plt.figure()
    plt.stem(df['n'],       df['theory_dBVrms'], linefmt='C0-', markerfmt='C0o', basefmt=" ",
             label='Theory (dBVrms)')
    plt.stem(df['n']+0.08,  df['meas_dBVrms'],   linefmt='C1-', markerfmt='C1s', basefmt=" ",
             label='Measurement (dBVrms)')
    plt.xlabel('Harmonic order n'); plt.ylabel('Amplitude [dBVrms]')
    ttl = title or f'{Path(csv_path).name} — {waveform.upper()}'
    if apply_rc_to_theory: ttl += ' (after RC, theory × |H|)'
    plt.title(ttl); plt.grid(True, alpha=0.35); plt.legend(); plt.tight_layout(); plt.show()

    return df  # handy to inspect errors inline if you want

# ===== Examples matching the lab comparisons =====
# 1) Sawtooth before filter (theory vs measurement, dBVrms)
compare_plot("fft_saw_in.csv", waveform='saw', apply_rc_to_theory=False, title="Sawtooth — input")

# 2) Triangle before filter
compare_plot("fft_triangle_in.csv", waveform='tri', apply_rc_to_theory=False, title="Triangle — input")

# 3) Sine before filter (only fundamental)
compare_plot("fft_sine_in.csv", waveform='sin', apply_rc_to_theory=False, title="Sine — input")

# 4) Sawtooth after RC (compare to theory multiplied by |H(jω)|)
compare_plot("fft_saw_after_rc.csv", waveform='saw', apply_rc_to_theory=True, title="Sawtooth — after RC")


FileNotFoundError: [Errno 2] No such file or directory: 'fft_saw_in.csv'

In [ ]:
# Apply smallest common axes to a pair of spectra and plot them separately
import numpy as np
import matplotlib.pyplot as plt

def plot_pair_smallest_axes(title, d1, label1, d2, label2, f0_hint=None, n_harm=10):
    (xmin, xmax), (ymin, ymax) = _common_axes_smallest(d1, d2, f0_hint=f0_hint, n_harm=n_harm)
    fig, axs = plt.subplots(2, 1, figsize=(9, 6), sharex=True, sharey=True)
    fig.suptitle(title)

    axs[0].plot(d1['freq_hz'], d1['dBVrms'], color='C0', lw=1)
    axs[0].set_title(label1)
    axs[0].grid(True, alpha=0.3)

    axs[1].plot(d2['freq_hz'], d2['dBVrms'], color='C1', lw=1)
    axs[1].set_title(label2)
    axs[1].grid(True, alpha=0.3)

    for ax in axs:
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_ylabel('Magnitude (dBVrms)')
    axs[-1].set_xlabel('Frequency (Hz)')

    plt.tight_layout()
    plt.show()

# Example usage elsewhere in the notebook (ensure d1/d2 exist):
# plot_pair_smallest_axes('Sine — RND vs Keysight', rnd_sine, 'RND', key_sine, 'Keysight', f0_hint=1000.0)

In [ ]:
# RND vs Keysight comparison (Sawtooth only) with smallest shared axes (dBVrms from Vrms)
import pandas as pd

def read_dbv(path):
    # Robust per-line parse: take last two numeric tokens per line as [freq, Vrms]
    freqs = []
    mags = []
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            if ';' not in line:
                continue
            parts = line.strip().split(';')
            nums = []
            for tok in reversed(parts):
                t = tok.replace(',', '.').strip()
                if not t:
                    continue
                try:
                    v = float(t)
                    nums.append(v)
                    if len(nums) == 2:
                        break
                except ValueError:
                    continue
            if len(nums) < 2:
                continue
            vrms = abs(nums[0])
            freq = nums[1]
            if not (np.isfinite(freq) and freq >= 0 and np.isfinite(vrms)):
                continue
            freqs.append(freq)
            mags.append(max(vrms, 1e-12))
    if not freqs:
        return pd.DataFrame({'freq_hz': [], 'dBVrms': []})
    freqs = np.array(freqs, dtype=float)
    mags = np.array(mags, dtype=float)
    dBV = 20.0 * np.log10(mags)
    df = pd.DataFrame({'freq_hz': freqs, 'dBVrms': dBV})
    df = df.sort_values('freq_hz').drop_duplicates(subset=['freq_hz']).reset_index(drop=True)
    return df

# Only Sawtooth comparison
rnd_saw_df = read_dbv('RND_lab_saw_fft.csv')
key_saw_df = read_dbv('trace_saw_fft.csv')

if rnd_saw_df.empty or key_saw_df.empty:
    print("Skipping RND vs Keysight sawtooth comparison due to missing data.")
else:
    plot_pair_smallest_axes('Sawtooth (~1 kHz) — RND vs Keysight (dBVrms)', rnd_saw_df, 'RND', key_saw_df, 'Keysight', f0_hint=1000.0)

NameError: name '_common_axes_smallest' is not defined

In [ ]:
# Common-axes helper: use smallest common x-range and smallest y-span (robust)
import numpy as np

def _common_axes_smallest(d1, d2, f0_hint=None, n_harm=10):
    # Expect pandas DataFrames with 'freq_hz' and 'dBVrms'
    f1 = np.asarray(d1['freq_hz'].to_numpy()) if 'freq_hz' in d1 else np.asarray([])
    y1 = np.asarray(d1['dBVrms'].to_numpy()) if 'dBVrms' in d1 else np.asarray([])
    f2 = np.asarray(d2['freq_hz'].to_numpy()) if 'freq_hz' in d2 else np.asarray([])
    y2 = np.asarray(d2['dBVrms'].to_numpy()) if 'dBVrms' in d2 else np.asarray([])

    # Preferred xmax by harmonics if f0_hint provided
    pref = n_harm * float(f0_hint) if (f0_hint is not None and f0_hint > 0) else None

    max1 = float(np.max(f1)) if f1.size else None
    max2 = float(np.max(f2)) if f2.size else None
    candidates = [x for x in [max1, max2, pref] if (x is not None and x > 0)]

    shared_xmax = min(candidates) if candidates else 0.0
    if shared_xmax <= 0.0:
        # fallback to any available max
        shared_xmax = max([x for x in [max1, max2] if x is not None] or [1.0])
    xmin = 0.0

    m1 = (f1 >= xmin) & (f1 <= shared_xmax) if f1.size else np.array([], dtype=bool)
    m2 = (f2 >= xmin) & (f2 <= shared_xmax) if f2.size else np.array([], dtype=bool)

    y1_slice = y1[m1] if (y1.size and np.any(m1)) else y1
    y2_slice = y2[m2] if (y2.size and np.any(m2)) else y2

    def span(arr):
        if arr.size:
            return float(np.nanmin(arr)), float(np.nanmax(arr)), float(np.nanmax(arr) - np.nanmin(arr))
        return -120.0, 20.0, 140.0

    y1_min, y1_max, span1 = span(y1_slice)
    y2_min, y2_max, span2 = span(y2_slice)

    if span1 <= span2:
        ymin, ymax = y1_min, y1_max
    else:
        ymin, ymax = y2_min, y2_max

    pad = 3.0
    return (xmin, shared_xmax), (ymin - pad, ymax + pad)


In [ ]:
# Generate first-10-harmonic tables (measurement vs theory)
from IPython.display import display
print("Sawtooth — input (trace_saw_fft.csv)")
display(table_10_harmonics('trace_saw_fft.csv', 'saw', F0=1000.0, N=10, apply_rc=False))
print("\nTriangle — input (trace_triangle_fft.csv)")
display(table_10_harmonics('trace_triangle_fft.csv', 'tri', F0=1000.0, N=10, apply_rc=False))
print("\nSine — input (trace_sine_fft.csv)")
display(table_10_harmonics('trace_sine_fft.csv', 'sin', F0=1000.0, N=10, apply_rc=False))
print("\nSawtooth — after RC (rc_fft_1khz.csv)")
display(table_10_harmonics('rc_fft_1khz.csv', 'saw', F0=1000.0, N=10, apply_rc=True))


# Optional: compact metric summaries for generator quality comparison (dBVrms)
rnd_sine  = summarize_metrics_dbv(load_fft_csv('RND_lab_sine_1khz_fft.csv'))
key_sine  = summarize_metrics_dbv(load_fft_csv('trace_sine_fft.csv'))
rnd_tri   = summarize_metrics_dbv(load_fft_csv('RND_lab_triangle_fft.csv'))
key_tri   = summarize_metrics_dbv(load_fft_csv('trace_triangle_fft.csv'))
rnd_saw   = summarize_metrics_dbv(load_fft_csv('RND_lab_saw_fft.csv'))
key_saw   = summarize_metrics_dbv(load_fft_csv('trace_saw_fft.csv'))


def print_metrics(name, m):
    print(f"{name}: f0={m['f0_hz']:.2f} Hz | Fundamental={m['fund_dBVrms']:.1f} dBVrms | THD={m['THD_dB']:.1f} dB | SFDR={m['SFDR_dB']:.1f} dB | Noise floor (med)={m['NoiseFloorMed_dBVrms']:.1f} dBVrms")


print("\nGenerator quality comparison (dBVrms metrics):")
print_metrics("RND — Sine", rnd_sine)
print_metrics("Keysight — Sine", key_sine)
print_metrics("RND — Triangle", rnd_tri)
print_metrics("Keysight — Triangle", key_tri)
print_metrics("RND — Sawtooth", rnd_saw)
print_metrics("Keysight — Sawtooth", key_saw)

## 3. Results (summary)
- All results are presented in dBVrms.
- Separate plots for RND vs Keysight are shown for sine, triangle, and sawtooth.
- Tables list the first 10 harmonics (measurement vs theory) and the RC case for sawtooth.

- Sawtooth: harmonics up to 10 kHz visible; amplitudes decay ≈ 1/n.
- Triangle: odd harmonics only; amplitudes decay ≈ 1/n².
- Sine: ideally only the fundamental; small extra lines may arise from leakage/noise/distortion.
- RC filter: attenuation increases with frequency (≈ −20 dB/dec above f_c); time waveform becomes smoother.

### Tables: first 10 harmonics (measurement vs theory, dBVrms)
Run the next cell to generate tables from the CSV files.


### RC attenuation at first 10 harmonics — theory vs measurement

In [ ]:
# Compute RC attenuation |H| at n*F0 (n=1..10) and compare to measured harmonics
import numpy as np
import pandas as pd
from IPython.display import display

# Build theory table: dBVrms for sawtooth after RC (theory × |H|)
N = 10
n = np.arange(1, N+1)
rc_mag_db = 20*np.log10(rc_mag(n*F0))
tho = theory_dBVrms(F0=F0, N=N, waveform='saw', apply_rc=True).copy()
tho = tho[['n', 'f_Hz', 'theory_dBVrms']]

# Load measured spectrum and pick nearest bins to n*F0
spec = read_scope_fft_vrms('rc_fft_1khz.csv')
meas = pick_harmonic_bins(spec, F0, N)

# Merge and compute delta
out = meas.merge(tho, left_on='n', right_on='n', how='left')
out['|H|(dB)'] = rc_mag_db
out['delta_meas_theory_dB'] = out['meas_dBVrms'] - out['theory_dBVrms']

# Pretty display
cols = ['n', 'f_bin_Hz', 'meas_dBVrms', 'theory_dBVrms', '|H|(dB)', 'delta_meas_theory_dB']
display(out[cols].round({'f_bin_Hz': 2, 'meas_dBVrms': 2, 'theory_dBVrms': 2, '|H|(dB)': 2, 'delta_meas_theory_dB': 2}))

## 4. Discussion
- Sawtooth vs triangle: Sawtooth includes all harmonics (1/n); triangle only odd harmonics with faster decay (1/n²). This reflects sharp edges vs smooth ramps in time domain.
- Sine: Ideally only the fundamental; extra lines indicate window leakage, noise floor, or source distortion.
- RC filter: The expected attenuation 20·log10 |H| broadly matches measurements. Deviations may stem from probe loading, FFT resolution, window choice, and dBVrms scaling.
- Phase: Increasing negative phase explains smoothing in the time domain (higher harmonics lag more than the fundamental).


## 5. Conclusions
- Fourier predictions for sawtooth, triangle, and sine match expected spectral trends and harmonic content.
- The RC low‑pass attenuates higher harmonics as expected and yields a smoother output in the time domain.
- Remaining discrepancies are consistent with practical limitations and FFT parameter choices.


## 6. References
- H. Hallenberg, Signal processing and communication systems, Yrgo, 2021.
- ELA24 — Laboratory: Frequency Analysis (Lab PM) and Fourier lab guide.
- Keysight/Tektronix oscilloscope FFT documentation (Hann window, dBVrms scaling).
- Standard RC low‑pass theory (jω method, Bode).

Use IEEE reference style in formal reports. Write in your own words; copying without proper citation is not allowed. Direct quotations are rare in technical reports.


In [ ]:
# Diagnostics: inspect structures used in comparisons
objs = {'rnd_sine': 'rnd_sine', 'key_sine': 'key_sine', 'rnd_tri': 'rnd_tri', 'key_tri': 'key_tri', 'rnd_saw': 'rnd_saw', 'key_saw': 'key_saw'}
for name, var in objs.items():
    try:
        o = globals().get(var, None)
        print(f"{name}: type={type(o)}")
        if isinstance(o, dict):
            print(f"  keys: {list(o.keys())[:6]}")
            if 'df' in o and hasattr(o['df'], 'head'):
                print(f"  df columns: {list(o['df'].columns)}  rows={len(o['df'])}")
        elif hasattr(o, 'head'):
            print(f"  df-like columns: {list(o.columns)}  rows={len(o)}")
    except Exception as e:
        print(f"{name}: error {e}")

In [ ]:
# Theory vs measurement plots (dBVrms)
# Uses compare_plot() to show first N harmonics vs theory for Keysight traces
compare_plot('trace_saw_fft.csv', waveform='saw', apply_rc_to_theory=False, title='Sawtooth — input (Keysight)')
compare_plot('trace_triangle_fft.csv', waveform='tri', apply_rc_to_theory=False, title='Triangle — input (Keysight)')
compare_plot('trace_sine_fft.csv', waveform='sin', apply_rc_to_theory=False, title='Sine — input (Keysight)')
# After RC: theory multiplied by |H(jω)|
compare_plot('rc_fft_1khz.csv', waveform='saw', apply_rc_to_theory=True, title='Sawtooth — after RC (Keysight)')

## Appendices

### Appendix A — Theory help (Fourier coefficients, dBVrms)

- Sawtooth (all harmonics):  $A_n = \dfrac{2A}{\pi n}$
- Triangle (odd harmonics): $A_n = \dfrac{8A}{\pi^2 n^2}$
- Sine: $A_1=A$, $A_{n>1}=0$
- RMS/dBV and RC low‑pass transfer function as defined in the Introduction.


### Appendix B — RC filter (theory)

$|H(f)| = \dfrac{1}{\sqrt{1+(f/f_c)^2}},\; \angle H(f) = -\arctan(f/f_c),\; f_c = \dfrac{1}{2\pi RC}$

Interpreted via Bode magnitude/phase; see parameters in the Experiment section.


### Appendix C — Data sources and formats

Sources compared in this report:
- RND 360-00002 Arbitrary Waveform Generator (AWG)
- Keysight oscilloscope built‑in Function Generator (FG)

All amplitudes are presented in dBVrms.

Data files and formats:
- RND_lab_*: FFTs of signals from the RND AWG
- trace_*_fft.csv: FFT traces with Keysight FG
- rc_fft_*: FFTs measured after the RC filter (when applicable)
- CSVs use semicolon as delimiter and comma as decimal; the last two numeric columns are [frequency (Hz), level]


### Appendix D — Alignment to assignments (ELA24)
- 1a. Sawtooth: measure first 10 harmonics and compare to theory.
- 1b. Triangle: measure 10 harmonics; odd n only in theory.
- 1c. Sine: fundamental only.
- 2. RC filter with sawtooth: compare output spectrum with theory × |H|.


### Appendix E — Image gallery (time and frequency domain)

Below are captured oscilloscope traces and FFT spectra. Filenames match raw data in the repo.

#### Time domain (inputs)
| Waveform | Image |
|---------|------|
| Sawtooth | ![Sawtooth (time)](saw_signal.png) |
| Triangle | ![Triangle (time)](triangle_signal.png) |
| Sine     | ![Sine (time)](sine_signal.png) |
| After RC (filtered sawtooth) | ![RC output (time)](rc_signal.png) |

#### FFT spectra (measured)
| Waveform | Image |
|---------|------|
| Sawtooth | ![Sawtooth FFT](saw_fft.png) |
| Triangle | ![Triangle FFT](triangle_fft.png) |
| Sine     | ![Sine FFT](sine_fft.png) |

#### RC filter (selected frequencies)
| Condition | Image |
|-----------|------|
| ~1 kHz (near fundamental) | ![RC ~1 kHz](rc_fft_1khz.png) |
| 10 kHz (> octave above f_c) | ![RC ~10 kHz](rc_fft_10khz.png) |
| 100 kHz (well above f_c) | ![RC ~100 kHz](rc_fft_100khz.png) |

(Extra) Higher‑frequency time domain:
| Trace | Image |
|------|------|
| 10 kHz time domain | ![RC 10 kHz time](rc_10khz.png) |
| 100 kHz time domain | ![RC 100 kHz time](rc_100khz.png) |

If images fail to render in PDF export, ensure the files are in the same folder as the notebook or embed them as base64.
